# Reconocimiento de señales de tráfico
### Máster Universitario en Ingeniería computacional y matemática - Área de inteligencia artificial
### Antonio González Hidalgo (agonzalezhidalgo@uoc.edu)

Usando una red neuronal convolucional.

Para la correcta funcionamiento de este notebook, el notebook debe de estar estructurado de la siguiente manera:

- ./dataset
- ./dataset/info.csv          Fichero que contiene los nombres de las señales correspondientes.
- ./dataset/train/Images      Conjunto de imágenes que formarán el entrenamiento de la red neuronal.
- ./dataset/test/info.csv.    Fichero que contiene las categórias de las imágenes de test.
- ./dataset/test/Images       Conjunto de imágenes que constituirán el test.

Para este notebook se ha usado el dataset _The German Traffic Sign Recognition Benchmark(GTSRB)_ http://benchmark.ini.rub.de/?section=gtsrb&subsection=news

In [1]:
import import_ipynb
import tfm_generic_functions as traffic

# https://docs.python.org/3/library/os.html
import os

# https://docs.scipy.org/doc/numpy/reference/
import numpy as np

# https://keras.io/models/model/
import keras

from keras import models

# Core Layers: https://keras.io/layers/core/
# Convolution Layers: https://keras.io/layers/convolutional/
from keras import layers

# https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

# https://keras.io/callbacks/
from keras.callbacks import ModelCheckpoint

importing Jupyter notebook from tfm_generic_functions.ipynb


Using TensorFlow backend.


In [2]:
# Obtenemos el directorio actual como trabajo.
ROOT_PATH = os.getcwd()

# Establecemos la dimensión de las imágenes.
IMG_SHAPE = (32, 32)

# Configurando la CNN
EPOCHS = 25
BATCH_SIZES = 128
ACTIVATION = 'selu'
OPTIMIZER = 'rmsprop'

print("Tamaño de las imágenes de entrada: ", IMG_SHAPE)
IMG_SHAPE_LEN = IMG_SHAPE[0] * IMG_SHAPE[1]
print("Vectorizando la entrada, sería de un tamaño: ", IMG_SHAPE_LEN)

# Obtenemos los paths de trabajo
labels_path = os.path.join(ROOT_PATH, "dataset_ger/labels.csv")
train_path = os.path.join(ROOT_PATH, "dataset_ger/train/Images")
test_info_path = os.path.join(ROOT_PATH, "dataset_ger/test/GT-final_test.csv")
test_path = os.path.join(ROOT_PATH, "dataset_ger/test/Images")

Tamaño de las imágenes de entrada:  (32, 32)
Vectorizando la entrada, sería de un tamaño:  1024


In [3]:
# Cargamos las imágenes de entrenamiento.
images_train, labels_train = traffic.readDataset(train_path, IMG_SHAPE, False)

# Convertimos las listas a array numpy de float32
np_images_train = np.asarray(images_train, dtype = np.float32)
np_labels_train = np.asarray(labels_train, dtype = np.int8)

# Recuperamos los nombres de las categorias. Los diferentes tipo de señales
# que se van a clasificar.
titles = traffic.read_csv(labels_path, ",")

# Se imprime información de los datos cargados.
traffic.print_size_dataset(images_train, labels_train, np_images_train, np_labels_train, "train")
print("Titles total: ", len(titles))

Total images (train):  39209
Total labels (train):  43
Images shape:  (39209, 32, 32, 3)
Labels shape:  (39209,)
Titles total:  43


In [4]:
# Cargamos las imágenes de entrenamiento.
images_test, labels_test = traffic.readDataset(test_path, IMG_SHAPE, False, False)

test_info  = traffic.read_csv(test_info_path, ";")
labels_test = traffic.get_class_id_array(test_info, 7)

# Convertimos las listas a array numpy de float32
np_images_test = np.asarray(images_test, dtype = np.float32)
np_labels_test = np.asarray(labels_test, dtype = np.int8)

# Se imprime información de los datos cargados.
traffic.print_size_dataset(images_test, labels_test, np_images_test, np_labels_test, "test")

Total images (test):  12630
Total labels (test):  43
Images shape:  (12630, 32, 32, 3)
Labels shape:  (12630,)


In [5]:
# Convertimos las labels de manera categórica
labels_categorical_train = keras.utils.to_categorical(np_labels_train)
labels_categorical_test = keras.utils.to_categorical(np_labels_test)

print("Ejemplo primera imagen de manera categórica: ", labels_categorical_train[0])

Ejemplo primera imagen de manera categórica:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
def get_keras_model(activation):
    # IMPLEMENTACIÓN RED NEURONAL
    # En Keras la envoltura para cualquier red neuronal se crea con la clase Sequential
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5, 5),
                            activation=activation, input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, (5, 5), activation=activation))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(len(set(labels_train)), activation='softmax'))
    return model


In [7]:
model = get_keras_model(ACTIVATION)

# Muestra la arquitectura de nuestra red neuronal
model.summary()

# Configurando el modelo de aprendijaze:
#  · loss, función para evaluar el grado de error entre salidas calculadas
#  · optimizador, función para calcular los pesos de los parámetros a partir de los datos de entrada
#  · metricas, para monitorizar el proceso de aprendizaje de la red.
model.compile(loss="categorical_crossentropy",
             optimizer=OPTIMIZER,
             metrics=['accuracy'])

# Entrenamiento del modelo
# - batch_size, indica el número de datos que se usan en cada actualización.
# - epochs, indica el número de veces que se usan todos los datos del proceso.
#model.fit(np_images_train, labels_categorical_train, batch_size=32, epochs=20)
model.fit(np_images_train, labels_categorical_train,
          batch_size = BATCH_SIZES,
          verbose = 2,
          epochs = EPOCHS,
          callbacks=[ModelCheckpoint('model_32_32_ger.h5', save_best_only = False)])

# Evaluación del modelo
test_loss, test_acc = model.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)  

In [8]:
datagen = ImageDataGenerator(featurewise_center=False,
                             featurewise_std_normalization=False, 
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                             shear_range=0.1,
                             rotation_range=10.,)

datagen.fit(np_images_test)

In [9]:
model_2 = get_keras_model(ACTIVATION)

model_2.compile(loss="categorical_crossentropy",
             optimizer=OPTIMIZER,
             metrics=['accuracy'])

model_2.fit_generator(datagen.flow(np_images_train, labels_categorical_train, batch_size=32),
                      steps_per_epoch = np_images_train.shape[0],
                      epochs = EPOCHS,
                      verbose = 2,
                      callbacks=[ModelCheckpoint('model_data_aug_32_32_ger.h5',save_best_only = False)])

Epoch 1/25
 - 2573s - loss: 0.4098 - acc: 0.8891
Epoch 2/25
 - 2552s - loss: 0.2366 - acc: 0.9408
Epoch 3/25
 - 2474s - loss: 0.2084 - acc: 0.9488
Epoch 4/25
 - 2566s - loss: 0.1931 - acc: 0.9530
Epoch 5/25
 - 2556s - loss: 0.1836 - acc: 0.9556
Epoch 6/25
 - 2648s - loss: 0.1767 - acc: 0.9577
Epoch 7/25
 - 1881s - loss: 0.1746 - acc: 0.9589
Epoch 8/25
 - 1470s - loss: 0.1711 - acc: 0.9597
Epoch 9/25
 - 1522s - loss: 0.1709 - acc: 0.9603
Epoch 10/25
 - 1499s - loss: 0.1694 - acc: 0.9610
Epoch 11/25
 - 1542s - loss: 0.1713 - acc: 0.9610
Epoch 12/25
 - 1536s - loss: 0.1705 - acc: 0.9612
Epoch 13/25
 - 1530s - loss: 0.1681 - acc: 0.9622
Epoch 14/25
 - 1543s - loss: 0.1702 - acc: 0.9619
Epoch 15/25
 - 1522s - loss: 0.1699 - acc: 0.9621
Epoch 16/25
 - 1523s - loss: 0.1682 - acc: 0.9621
Epoch 17/25
 - 1543s - loss: 0.1685 - acc: 0.9627
Epoch 18/25
 - 1548s - loss: 0.1696 - acc: 0.9624
Epoch 19/25
 - 1574s - loss: 0.1714 - acc: 0.9626
Epoch 20/25
 - 1580s - loss: 0.1706 - acc: 0.9630
Epoch 21/

In [10]:
# Evaluación del modelo
test_loss, test_acc = model_2.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

12630/12630 [==============================] - 5s 403us/step
Test loss: 15.23345929626803
Test accuracy: 0.037925574030087096
